## Import library

In [58]:
import pandas as pd
import numpy as np 
import sklearn 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector



## Read data 

In [44]:
data_path = '/Users/caobai/code/JaneChutingCao/lewagon_project/raw_data/train_values.csv'
label_path = '/Users/caobai/code/JaneChutingCao/lewagon_project/raw_data/train_labels.csv'
a = pd.read_csv(data_path)
b = pd.read_csv(label_path)

df = a.merge(b, on ='building_id')


In [45]:
df = df
df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [46]:
X = df.drop(['damage_grade'], axis = 1)
y = df['damage_grade']


## Feature scaling 

### Selecting text columns 

In [47]:
text_columns = [name for col, name in zip(X, X.columns) if X[col].dtypes =='object']
text_columns


['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

### Selecting numeric columns

In [6]:
num_columns = [name for col, name in zip(X, X.columns) if X[col].dtypes == 'int64']
len(num_columns)


30

In [12]:
nonbinary_columns = num_columns[:8]
nonbinary_columns.append('count_families')
nonbinary_columns.remove('has_superstructure_adobe_mud')
len(nonbinary_columns)

8

In [13]:
binary_columns  = [item for item in num_columns if item not in nonbinary_columns]
len(binary_columns)

22

### Making feature scaling pipeline 

In [14]:
# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown = 'ignore')


In [15]:
# Use minmax scaler on nonbinary features
# Use Robust scaler on columns with outliers 
mm_scaler = MinMaxScaler()
rb_scaler = RobustScaler()
nonbinary_columns_no_outliers = nonbinary_columns[:3]
nonbinary_columns_with_outliers = nonbinary_columns[3:]

In [16]:
preporcessor = ColumnTransformer([
    ('mm_scaler', mm_scaler, nonbinary_columns_no_outliers),
    ('rb_scaler', rb_scaler, nonbinary_columns_with_outliers),
    ('cat_transformer', cat_transformer, text_columns)
], remainder = 'passthrough')

In [17]:
preporcessor



ColumnTransformer(remainder='passthrough',
                  transformers=[('mm_scaler', MinMaxScaler(),
                                 ['geo_level_1_id', 'geo_level_2_id',
                                  'geo_level_3_id']),
                                ('rb_scaler', RobustScaler(),
                                 ['count_floors_pre_eq', 'age',
                                  'area_percentage', 'height_percentage',
                                  'count_families']),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['land_surface_condition', 'foundation_type',
                                  'roof_type', 'ground_floor_type',
                                  'other_floor_type', 'position',
                                  'plan_configuration',
                                  'legal_ownership_status'])])

In [20]:
X_prec = preporcessor.fit_transform(X)
X_prec.shape


(260601, 68)